# Source
https://www.kaggle.com/carrie1/ecommerce-data

# Context
Typically e-commerce datasets are proprietary and consequently hard to find among publicly available data. However, The UCI Machine Learning Repository has made this dataset containing actual transactions from 2010 and 2011. The dataset is maintained on their site, where it can be found by the title "Online Retail".

# Content
"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."

# Acknowledgements
Per the UCI Machine Learning Repository, this data was made available by Dr Daqing Chen, Director: Public Analytics group. chend '@' lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.

# Inspiration
Analyses for this dataset could include time series, clustering, classification and more.

1. Data Preparation
2. Exploring the content of variables
    * Negative quantity and unitprice
    * Country
    * Customer
    * Product
3. Trending
    * New Customers by month
    * Sales by month

# 1. Data Preparation

## Import the library and data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import datetime as dt
from plotly.offline import init_notebook_mode,iplot
import warnings


warnings.simplefilter(action='ignore', category=FutureWarning)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Any results you write to the current directory are saved as output.# 1. Data Preparation

In [ ]:
df = pd.read_csv('/kaggle/input/ecommerce-data/data.csv',encoding = "ISO-8859-1")
df.head()

Check if there is any missing data

In [ ]:
df.count()

It seems that there're many rows which **CustomerID** is missing and a few row with (product) ** Description** is missing. I'll keep in mind about this but not filter it out yet.

Next, I'm going to check the type of data in each columns

In [ ]:
df.dtypes

Convert **InvoiceDate** to datetime

In [ ]:
df['InvoiceDate'] = df['InvoiceDate'].astype('datetime64[ns]')

#Convert date to year-month
df['InvoiceMonth'] = df['InvoiceDate'].dt.strftime('%Y-%m')

# 2. Exploring the content of variables

## 2.1 Negative Quantity and UnitPrice

Take a quick look in numerical columns

In [ ]:
df.describe()

The **min** in each column is negative value which probably means a refund order or the data is incorrect but I assume that it's a refund order. I'll check more about it, start with negative **Quantity**.

In [ ]:
df[df['Quantity']<0].head()

The **InvoiceNo** has prefix **C** which probably means canceled order. I'll look into **CustomerID** 17584 data if we find something else.

In [ ]:
df[df['CustomerID']==17548]

The **InvoiceNo** 550755 and C552049 are exactly the same in quantity and products, but the **InvoiceDate** of **C** is over a month later. So I think **InvoiceNo** with C in prefix is refund order.

Another case is **InvoiceNo** C536391 which has no matched order but if we look at the **InvoiceDate** and **index**, this refund order is very early in dataset, so I think it refund for the order that occured before the company collected the data.

Let's see if there're a lot of refunds.

In [ ]:
cnt_order = df[df['Quantity']>0]['InvoiceNo'].nunique()
cnt_refund = df[df['Quantity']<0]['InvoiceNo'].nunique()


print("Total Orders : ",cnt_order)
print("Total Refund Order : ",cnt_refund)
print("%Refund : ",cnt_refund/(cnt_order)*100,"%")

Number of refund order is aroud 25%! We should dig deeper to find why the %refund is so high (may be I was wrong about refund).

I create **RefundFlg** to make in easier when we want to filter it.

In [ ]:
df['RefundFlg'] = df['Quantity']<0

Finding out which products has the most %refund

In [ ]:
prod_order = df[df['Quantity']>0].groupby(['StockCode','Description']).InvoiceNo.nunique().sort_values(ascending = False).reset_index()
prod_order = prod_order.rename(columns = {'InvoiceNo' : 'TotalOrder'})
prod_order_refund = df[df['Quantity']<0].groupby(['StockCode','Description']).InvoiceNo.nunique().sort_values(ascending = False).reset_index()
prod_order_refund = prod_order_refund.rename(columns = {'InvoiceNo' : 'TotalRefundOrder'})

join_prod_order = prod_order.merge(prod_order_refund,left_on = ["StockCode","Description"],right_on = ["StockCode","Description"],how = 'left')
join_prod_order['%Refund'] = join_prod_order['TotalRefundOrder']/join_prod_order['TotalOrder']*100
join_prod_order = join_prod_order.sort_values(by = 'TotalRefundOrder',ascending = False)
join_prod_order.head()

**M** doesn't seem like the name of product, may be there is something is hidden inside the data.

This time I'll sort by %refund but select only product that has at least 10 refund order.

In [ ]:
join_prod_order = join_prod_order.sort_values(by = '%Refund',ascending = False)
join_prod_order[join_prod_order['TotalRefundOrder']>=10].head(10)

The top 4 doesn't seem like the name of products and **TotalRefundOrder** is higher that **TotalOrder**! From the descriptions, I think in this case, it is about additional fee or discounted fee.

Even the top 4 is not the problems, other products in top 10 seem to be problems and should find out why these product has high %refund.

Next, I'll try to recalculate %refund again by excluding the non-product (top 4 in above table)

In [ ]:
prod_only_order = join_prod_order[~join_prod_order['StockCode'].isin(['AMAZONFEE','S','BANK CHARGES','M'])]
TotalRefundProdOrder = prod_only_order['TotalRefundOrder'].sum()
TotalReProdOrder = prod_only_order['TotalOrder'].sum()
print("%Refund : ",TotalRefundProdOrder/TotalReProdOrder*100,"%")

%Refund is 1.68%. It makes a lot more sense!

> **Comment on this part:**
    It's okay to use C as prefix to define that this order is refund but it would be a lot better if it can specify which order is refunded. For example, **InvoiceNo** : C1234 is the refund order of **InvoiceNo** : 1234 or you can create a new column that tell which ordered is refunded (the value of this column in normal order is null). 

Next let's see the data with negative **UnitPrice**.

In [ ]:
df[df['UnitPrice']<0].head()

The prefix of **InvoiceNo** is A in thist case and **Description** tell us that it means bad debt. So the company might give customers credit and some of them didn't pay back and becomes bad debt.

Creating **Net** column for easier calculation sales in the future. The **Net** is calculated from **Quantity** x **UnitPrice**. 

Note : Keep in mind that the refund order has negative **Net** value

In [ ]:
df['Net'] = df['Quantity']*df['UnitPrice']

## 2.2 Country

In [ ]:
sales_by_country = df.groupby(['Country']).Net.sum().sort_values(ascending = False).reset_index()
TotalSales = sales_by_country['Net'].sum()


sales_by_country['% of total sales'] = sales_by_country['Net']/TotalSales
#Top 5 countries by sales
sales_by_country.head()

Most of the sales are from UK which is around 84% of total sales (Company is based in UK)


## 2.3 Customers

In this part I'd like to learn more about the customers. I'll start with calculating:
* Average per Order (AOV)
* Repeat Customers : Customers who has more than 1 orders

Note : 
* There're many rows that **CustomerID** is missing so I'll filter out in this part.
* Assume that there is no refund and bad debt to simplify calculation

In [ ]:
df_wt_cust = df[(df['CustomerID'].notnull()) & (df['Quantity']>0)]

order_net = df_wt_cust.groupby(['InvoiceNo']).Net.sum()
aov = order_net.mean()
plt.hist(order_net,bins=1000)
plt.xlim(0,2000)
plt.xlabel("Order value")
plt.ylabel("Number of orders")
print("AOV : ",aov)

The aov is 480. Next I'm going to find the %repeat customers.

From the last part, we learned that there're refunded orders, so I'll calculated number of orders per customer by count total orders and subtract by total refund order by each customer

In [ ]:
#Repeat Customers

#Exlude non-product rows

df_for_rpt_cust = df_wt_cust.copy()
df_for_rpt_cust = df_for_rpt_cust[~df_for_rpt_cust['StockCode'].isin(['AMAZONFEE','S','BANK CHARGES','M'])]

cust_wt_total_order = df_for_rpt_cust[df_for_rpt_cust['Net']>0].groupby(['CustomerID']).InvoiceNo.nunique().reset_index()
cust_wt_total_order = cust_wt_total_order.rename(columns = {'InvoiceNo' : 'TotalOrder'})

cust_wt_total_refund_order = df_for_rpt_cust[df_for_rpt_cust['Net']<0].groupby(['CustomerID']).InvoiceNo.nunique().reset_index()
cust_wt_total_refund_order = cust_wt_total_refund_order.rename(columns = {'InvoiceNo' : 'TotalRefundOrder'})

join_cust_wt_total_order = cust_wt_total_order.merge(cust_wt_total_refund_order,left_on = 'CustomerID',right_on='CustomerID',how = 'left')
# convert null to 0
join_cust_wt_total_order['TotalRefundOrder'] = np.where(join_cust_wt_total_order['TotalRefundOrder'].isnull(),0,join_cust_wt_total_order['TotalRefundOrder'])
join_cust_wt_total_order['TotalSuccessOrder'] = join_cust_wt_total_order['TotalOrder']-join_cust_wt_total_order['TotalRefundOrder']

join_cust_wt_total_order['RepeatFlg'] = join_cust_wt_total_order['TotalSuccessOrder']>=2

CntCustomer = join_cust_wt_total_order.CustomerID.nunique()
CntRepeatCustomer = join_cust_wt_total_order[join_cust_wt_total_order['RepeatFlg']==True].CustomerID.nunique()
PctRepeatCustomer = CntRepeatCustomer/CntCustomer

print("%Repeat Customer : ",PctRepeatCustomer*100,"%")

plt.hist(join_cust_wt_total_order['TotalSuccessOrder'],bins = 100)
plt.xlim(0,30)
plt.xlabel("Number of Orders")
plt.ylabel("Number of Customers")
plt.show()



%Repeat Customer is 65.5% which is pretty high and many customers keep coming back multiple times.

Let's see which products that make customers keep coming back

In [ ]:
df_wt_cust.groupby(['CustomerID','Description']).InvoiceNo.nunique().sort_values(ascending = False).reset_index().head(10)

From the top 10 products, we learn that there're customers who buy a lot of products and a lot of orders. These customers might buy this orders to sell in their area (just guessing)

The previous data table makes me curious about sales concentration in customers. If most of the sales are from just a few of  customers, I think it's risky for the company because when these customer gone, all of the sales gone.

The next thing I'm going to find is the number of customers that make 80% sales of the company. I want to make sure that this number is not too low. I'll select only sales in last 3 months (from 9Sep2011)

In [ ]:
sales_by_cust = df_wt_cust[df_wt_cust['InvoiceDate']>dt.date(2011,9,9)].groupby('CustomerID').Net.sum().sort_values(ascending=False).reset_index()
TotalSales = sales_by_cust.Net.sum()
sales_by_cust['%TotalSales'] = sales_by_cust['Net']/TotalSales


def cust_concentration(threshold):

    cnt_cust = 0
    accm_pct = 0

    for index, row in sales_by_cust.iterrows():
        if accm_pct ==0:
            accm_pct = row['%TotalSales']
        else:
            accm_pct = accm_pct+row['%TotalSales']
        cnt_cust = cnt_cust+1
        
        if accm_pct>=threshold:
            return cnt_cust


print("70% of sales are from ",cust_concentration(0.7)," customers")        
print("80% of sales are from ",cust_concentration(0.8)," customers")
print("90% of sales are from ",cust_concentration(0.9)," customers")
print("100% of sales are from ",sales_by_cust.CustomerID.nunique()," customers")



80% of sales are from 956 customers which is pretty high number. So even some customers stop buying, it won't affect the company mmuch.

## 2.4 Products

It's the same as customer, I want to make sure that company sales don't rely to much on some products. So I'm going to find product sales concentration, the approach is the same as customers.

Note : Calculation in this case is not related to the **CustomerID**, so I'm going to use all data to calculate (even **CustomerID** is missing).

In [ ]:
sales_by_prod = df[df['InvoiceDate']>dt.date(2011,9,9)].groupby('StockCode').Net.sum().sort_values(ascending=False).reset_index()
TotalSalesAll = sales_by_prod.Net.sum()
sales_by_prod['%TotalSales'] = sales_by_prod['Net']/TotalSalesAll

def prod_concentration(threshold):

    cnt_prod = 0
    accm_pct = 0

    for index, row in sales_by_prod.iterrows():
        if accm_pct ==0:
            accm_pct = row['%TotalSales']
        else:
            accm_pct = accm_pct+row['%TotalSales']
        cnt_prod = cnt_prod+1
        
        if accm_pct>=threshold:
            return cnt_prod


print("70% of sales are from ",prod_concentration(0.7)," products")        
print("80% of sales are from ",prod_concentration(0.8)," products")
print("90% of sales are from ",prod_concentration(0.9)," products")
print("100% of sales are from ",sales_by_prod.StockCode.nunique()," products")

The number of products is high enough, so the company don't rely to much on few products.

# 3. Trending

In this part, I want to see if company is still okay by looking at trend in sales and customers.

## 3.1 Number of new customers by month

In [ ]:

df_for_trend = df.copy()

#Define new customer

cust_wt_fst_order = df_for_trend.groupby(['CustomerID']).InvoiceDate.min().reset_index()
cust_wt_fst_order = cust_wt_fst_order.rename(columns = {"InvoiceDate" : "FirstInvoiceDate"})

df_for_trend = df_for_trend.merge(cust_wt_fst_order,left_on = 'CustomerID',right_on = 'CustomerID',how = 'left')

df_for_trend['FirstOrderFlg'] = (df_for_trend['InvoiceDate'] == df_for_trend['FirstInvoiceDate'])

df_new_cust = df_for_trend[df_for_trend['FirstOrderFlg']==True][['CustomerID','InvoiceMonth']].drop_duplicates()

agg_df_new_cust = df_new_cust.groupby("InvoiceMonth").CustomerID.nunique().reset_index()
plt.bar(agg_df_new_cust['InvoiceMonth'],agg_df_new_cust['CustomerID'])
plt.xticks(rotation=90)
plt.xlabel("InvoiceMonth")
plt.ylabel("Number of New Customers")
plt.title('Number of New Customers by month')
plt.show()


In [ ]:
agg_df_cust = df_for_trend.groupby("InvoiceMonth").CustomerID.nunique().reset_index()

plt.bar(agg_df_cust['InvoiceMonth'],agg_df_cust['CustomerID'])
plt.xticks(rotation=90)
plt.xlabel("InvoiceMonth")
plt.ylabel("Number of Customers")
plt.title('Number of Customers by month')

plt.show()

## 3.2 Sales by month

In [ ]:
agg_sales = df_for_trend.groupby('InvoiceMonth').Net.sum().reset_index()

plt.bar(agg_df_cust['InvoiceMonth'],agg_df_cust['CustomerID'])
plt.xticks(rotation=90)
plt.xlabel("InvoiceMonth")
plt.ylabel("Sales")
plt.title('Sales by month')
plt.show()

From the chart above, it seems that the data on Dec2011 is not complete, so we'll look only data from Dec10-Nov11 (12 months).

Number of new customers is clearly in downtrend during Mar11-Aug11 and higher in Sep11-Nov11 because of high season period (data from sales by month that sales spike during the end of the year). One thing that company should be concern is customer acquisition because customer has their own lifecycle, so one day they'll churn anyway. There're many things that company can do such as increasing budget for acquiring new customers, analyzing the source of new customers to improve stretegies.

Another thing we learn from chart is seasonality is sales. Because the products of the company is gift, it depends on holidies or ceremonies (as we can see from sales at the end of the year). So the company should always prepare in inventory and manpower before peak season.